### Import Libraries

In [2]:
!pip install keras

In [5]:
!pip install plotly

  Using cached retrying-1.3.3.tar.gz (10 kB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11434 sha256=9ada3f8f5f0ac2d58c47de19e42a2862a8676f6699da3292cc9ea017c58b2d7f
  Stored in directory: c:\users\sahand\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [24]:
!pip install tensorflow

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as tf

### Data

In [2]:
data = pd.read_csv("simulation_Data_2020.csv")
data.head()

,Date_Time,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
0,2020/01/01 01:00:00,11.60,-7.45,19.048198,7.31,0.0,0.2,7.31
1,2020/01/01 02:00:00,11.60,-7.56,19.160009,7.36,0.0,0.2,7.36
2,2020/01/01 03:00:00,11.58,-8.46,20.043659,7.63,0.0,0.2,7.63
3,2020/01/01 04:00:00,11.54,-9.18,20.710560,7.82,0.0,0.2,7.82
4,2020/01/01 05:00:00,11.49,-9.71,21.201178,7.97,0.0,0.2,7.97


### Data visulization

In [3]:
power_heating = data[['power_heating']]
power_heating_data = go.Scatter(x = data['Date_Time'], y = data['power_heating'])
layout = go.Layout(title ='Power heating time series', xaxis = dict(title='Date'), yaxis = dict(title='Power heating'))

fig = go.Figure(data = [power_heating_data], layout = layout)
fig.write_html('Scatter_Power_Heating.html', auto_open = True)

### Normalization

In [4]:
min_max_scaler = MinMaxScaler()
norm_data = min_max_scaler.fit_transform(power_heating.values)
norm_data
norm_data_df = pd.DataFrame(norm_data)
norm_data_df.head()

,0
0,0.791126
1,0.796537
2,0.825758
3,0.846320
4,0.862554


In [5]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [6]:
past_history = 5
future_target = 0

TRAIN_SPLIT = int(len(norm_data) * 0.8)


x_train, y_train = univariate_data(norm_data,
                                   0,
                                   TRAIN_SPLIT,
                                   past_history,
                                   future_target)

x_test, y_test = univariate_data(norm_data,
                                 TRAIN_SPLIT,
                                 None,
                                 past_history,
                                 future_target)

In [7]:
num_units = 64
learning_rate = 0.0001
activation_function = 'sigmoid'
adam = Adam(lr=learning_rate)
loss_function = 'mse'
batch_size = 5
num_epochs = 50

# Initialize the RNN
model = Sequential()
model.add(LSTM(units = num_units, activation=activation_function, input_shape=(None, 1)))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer=adam, loss=loss_function)

In [8]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size=batch_size,
    epochs=num_epochs,
    shuffle=False
)

Epoch 1/50
1261/1261 [==============================] - 9s 6ms/step - loss: 0.1112 - val_loss: 0.0564
Epoch 2/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.1295 - val_loss: 0.0502
Epoch 3/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.1143 - val_loss: 0.0456
Epoch 4/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.0973 - val_loss: 0.0406
Epoch 5/50
1261/1261 [==============================] - 6s 4ms/step - loss: 0.0864 - val_loss: 0.0371
Epoch 6/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.0708 - val_loss: 0.0340
Epoch 7/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.0614 - val_loss: 0.0316
Epoch 8/50
1261/1261 [==============================] - 6s 4ms/step - loss: 0.0542 - val_loss: 0.0294
Epoch 9/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.0477 - val_loss: 0.0287
Epoch 10/50
1261/1261 [==============================] - 5s 4ms/step - loss: 0.044

In [10]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))
epochs_list = [*range(0, 50, 1)]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = epochs_list, y = loss, mode = 'lines', name = 'Loss'))
fig2.add_trace(go.Scatter(x = epochs_list, y = val_loss, mode = 'lines', name = 'Loss Val'))
fig2.update_layout(
    title="Power heating Loss VS Loss Val",
    xaxis_title="Epochs",
    yaxis_title="Loss and Loss Val",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig2.write_html('Power heating Loss VS Los Val.html', auto_open = True)

In [11]:
original = pd.DataFrame(min_max_scaler.inverse_transform(y_test))
predictions = pd.DataFrame(min_max_scaler.inverse_transform(model.predict(x_test)))

In [12]:
predictions

,0
0,5.227692
1,5.599319
2,1.872113
3,2.051075
4,3.216311
...,...
1742,3.678045
1743,3.770712
1744,3.864950
1745,3.966822


In [13]:
original

,0
0,8.31
1,1.56
2,2.20
3,4.54
4,3.28
...,...
1742,5.01
1743,5.13
1744,5.26
1745,5.46


In [16]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = original.index, y = original[0], mode = 'lines', name = 'Test Data'))
fig3.add_trace(go.Scatter(x = predictions.index, y = predictions[0], mode = 'lines', name = 'Predictions'))
fig3.update_layout(
    title="Original VS Predictions",
    xaxis_title="Days",
    yaxis_title="Original and Predictions",
#     legend_title="Legend Title",
)
# layout2 = go.Layout(title ='Power heating Loss VS Loss Val')
fig3.write_html('Original VS Predictions.html', auto_open = True)